In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder

# ============================
# 0️⃣ Load & prepare data
# ============================
merged_train_df = pd.read_csv('merged_train_df.csv')
test_df = pd.read_csv("test_df.csv")

merged_train_df['date'] = pd.to_datetime(merged_train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

merged_train_df = merged_train_df.sort_values("date").reset_index(drop=True)
test_df = test_df.sort_values("date").reset_index(drop=True)

merged_train_df['store_nbr'] = merged_train_df['store_nbr'].astype(int)
test_df['store_nbr'] = test_df['store_nbr'].astype(int)

# ============================
# 1️⃣ ACF helper
# ============================
def acf_features(series, seasonal_lag=7):
    s = series.dropna()
    if len(s) < 15:
        return pd.Series({k: 0.0 for k in [
            "first_acf_original","sumsq_acf_original",
            "first_acf_diff1","sumsq_acf_diff1",
            "first_acf_diff2","sumsq_acf_diff2",
            "seasonal_acf"
        ]})
    try:
        acfs = [s.autocorr(lag=i) if not pd.isna(s.autocorr(lag=i)) else 0.0 for i in range(1,11)]
        first_acf = acfs[0]
        diff1 = s.diff().dropna()
        acfs_diff1 = [diff1.autocorr(lag=i) if len(diff1) > i else 0.0 for i in range(1,11)]
        first_acf_diff1 = acfs_diff1[0] if len(acfs_diff1)>0 else 0.0
        diff2 = diff1.diff().dropna()
        acfs_diff2 = [diff2.autocorr(lag=i) if len(diff2) > i else 0.0 for i in range(1,11)]
        first_acf_diff2 = acfs_diff2[0] if len(acfs_diff2)>0 else 0.0
        seasonal_acf = s.autocorr(lag=seasonal_lag) if len(s) > seasonal_lag else 0.0

        out = {
            "first_acf_original": first_acf,
            "sumsq_acf_original": np.sum(np.square(acfs)),
            "first_acf_diff1": first_acf_diff1,
            "sumsq_acf_diff1": np.sum(np.square(acfs_diff1)),
            "first_acf_diff2": first_acf_diff2,
            "sumsq_acf_diff2": np.sum(np.square(acfs_diff2)),
            "seasonal_acf": seasonal_acf
        }
    except:
        out = {k:0.0 for k in [
            "first_acf_original","sumsq_acf_original",
            "first_acf_diff1","sumsq_acf_diff1",
            "first_acf_diff2","sumsq_acf_diff2",
            "seasonal_acf"
        ]}
    return pd.Series(out)

# ============================
# 2️⃣ Compute ACF table correctly - FIXED
# ============================
print(">>> Computing ACF features per store...")

# Compute ACF features per store
acf_table_raw = merged_train_df.groupby("store_nbr")["sales"].apply(acf_features)

# FIXED: Properly convert to DataFrame by unstacking
acf_table = acf_table_raw.unstack().reset_index()

print(">>> ACF table shape:", acf_table.shape)
print(">>> ACF table columns:", acf_table.columns.tolist())
print(">>> ACF table head:")
print(acf_table.head())

# Verify we have the expected columns
expected_acf_cols = ['first_acf_original','sumsq_acf_original','first_acf_diff1','sumsq_acf_diff1',
                     'first_acf_diff2','sumsq_acf_diff2','seasonal_acf']
print(">>> Expected ACF columns present:", all(col in acf_table.columns for col in expected_acf_cols))

# ============================
# 3️⃣ Feature engineering with proper merge
# ============================
def feature_engineering(df, acf_lookup):
    df = df.copy()
    df['store_family'] = df['store_nbr'].astype(str) + "_" + df['family'].astype(str)
    df['promo_holiday'] = df['onpromotion'] * df['isHoliday']
    df['oil_price_diff'] = df.groupby('store_nbr')['oil_price'].diff().fillna(0)
    df['oil_price_rolling_7'] = df.groupby('store_nbr')['oil_price'].rolling(7, min_periods=1).mean().reset_index(0, drop=True).bfill()
    df['oil_price_7d_avg'] = df['oil_price_rolling_7']
    df['onpromotion_7d_sum'] = df.groupby('store_nbr')['onpromotion'].rolling(7, min_periods=1).sum().reset_index(0, drop=True).fillna(0)
    df['salary_promo'] = df['salary_day_impact'] * df['onpromotion']

    print(">>> Merging ACF features...")
    print(">>> Before merge - df shape:", df.shape)
    print(">>> ACF lookup shape:", acf_lookup.shape)
    
    df = df.merge(acf_lookup, on="store_nbr", how="left")
    print(">>> After merge - df shape:", df.shape)

    acf_cols = ['first_acf_original','sumsq_acf_original','first_acf_diff1','sumsq_acf_diff1',
                'first_acf_diff2','sumsq_acf_diff2','seasonal_acf']
    
    for col in acf_cols:
        if col not in df.columns:
            print(f"[WARN] {col} missing in merged df, filling with 0")
            df[col] = 0.0
        else:
            print(f"[INFO] {col} successfully merged, non-null count: {df[col].notna().sum()}")
            df[col] = df[col].fillna(0.0)

    return df

print(">>> Adding features to train and test...")
train_fe = feature_engineering(merged_train_df, acf_table)
test_fe = feature_engineering(test_df, acf_table)

# ============================
# 4️⃣ Encode categoricals
# ============================
exog_cols = ['onpromotion','oil_price','isHoliday','earthquake_impact','salary_day_impact',
             'promo_holiday','oil_price_diff','oil_price_rolling_7','oil_price_7d_avg',
             'onpromotion_7d_sum','salary_promo',
             'first_acf_original','sumsq_acf_original','first_acf_diff1','sumsq_acf_diff1',
             'first_acf_diff2','sumsq_acf_diff2','seasonal_acf']

cat_cols = ["city","state","family","type","store_family"]

cat_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    combined_values = pd.concat([train_fe[col].astype(str), test_fe[col].astype(str)]).unique()
    le.fit(combined_values)
    train_fe[col] = le.transform(train_fe[col].astype(str))
    test_fe[col] = le.transform(test_fe[col].astype(str))
    cat_encoders[col] = le
    exog_cols.append(col)

# ============================
# 5️⃣ Train SARIMAX per store
# ============================
predictions = []

common_stores = set(train_fe['store_nbr'].unique()) & set(test_fe['store_nbr'].unique())

for store_num in sorted(common_stores):
    train_store_df = train_fe[train_fe['store_nbr']==store_num].sort_values('date')
    test_store_df = test_fe[test_fe['store_nbr']==store_num].sort_values('date')

    # Use the correct sales column
    y_train = train_store_df['sales']

    if len(y_train) < 10:
        y_pred = pd.Series([y_train.mean()] * len(test_store_df))
        out = test_store_df[["id"]].copy()
        out["sales"] = y_pred
        predictions.append(out)
        continue

    available_exog = [c for c in exog_cols if c in train_store_df.columns and c in test_store_df.columns]
    if len(available_exog) == 0:
        y_pred = pd.Series([y_train.mean()] * len(test_store_df))
        out = test_store_df[["id"]].copy()
        out["sales"] = y_pred
        predictions.append(out)
        continue

    exog_train = train_store_df[available_exog].fillna(0)
    exog_test = test_store_df[available_exog].fillna(0)

    try:
        model = SARIMAX(y_train, order=(1,1,1), seasonal_order=(0,0,0,0),
                        exog=exog_train, enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False, maxiter=50)
        y_pred = model_fit.forecast(steps=len(exog_test), exog=exog_test)
    except:
        y_pred = pd.Series([y_train.mean()] * len(exog_test))

    y_pred = np.maximum(y_pred, 0)
    out = test_store_df[["id"]].copy()
    out["sales"] = y_pred.values
    predictions.append(out)

final_preds = pd.concat(predictions, ignore_index=True)

# ============================
# 6️⃣ Submission
# ============================
submission_df = final_preds.copy()
submission_df['sales'] = submission_df['sales'].astype(float)
submission_df['sales'] = submission_df['sales'].fillna(0)
submission_df.to_csv("my_submission.csv", index=False)
print(">>> Submission saved to 'my_submission.csv'")

# ============================
# 7️⃣ Debugging output
# ============================
print("\n>>> DEBUGGING INFO:")
print(">>> Sample of ACF values for first few stores:")
print(acf_table.head(10))
print("\n>>> Summary statistics for ACF features:")
for col in ['first_acf_original','sumsq_acf_original','seasonal_acf']:
    if col in acf_table.columns:
        print(f"{col}: mean={acf_table[col].mean():.4f}, std={acf_table[col].std():.4f}, min={acf_table[col].min():.4f}, max={acf_table[col].max():.4f}")

>>> Computing ACF features per store...
>>> ACF table shape: (54, 8)
>>> ACF table columns: ['store_nbr', 'first_acf_original', 'sumsq_acf_original', 'first_acf_diff1', 'sumsq_acf_diff1', 'first_acf_diff2', 'sumsq_acf_diff2', 'seasonal_acf']
>>> ACF table head:
   store_nbr  first_acf_original  sumsq_acf_original  first_acf_diff1  \
0          1           -0.060393            0.028752        -0.521795   
1          2           -0.067574            0.056076        -0.527110   
2          3           -0.089117            0.125725        -0.534634   
3          4           -0.088535            0.158816        -0.531400   
4          5           -0.095750            0.112813        -0.524814   

   sumsq_acf_diff1  first_acf_diff2  sumsq_acf_diff2  seasonal_acf  
0         0.303261        -0.674733         0.529303     -0.072911  
1         0.326922        -0.679632         0.545572      0.043151  
2         0.398940        -0.699452         0.658790     -0.065157  
3         0.544911     

/Users/donaldaadithiyan/Desktop/Uni/Sem 5/Time Series-Assignment/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/donaldaadithiyan/Desktop/Uni/Sem 5/Time Series-Assignment/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/donaldaadithiyan/Desktop/Uni/Sem 5/Time Series-Assignment/venv/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/donaldaadithiyan/Desktop/Uni/Sem 5/

>>> Submission saved to 'my_submission.csv'

>>> DEBUGGING INFO:
>>> Sample of ACF values for first few stores:
   store_nbr  first_acf_original  sumsq_acf_original  first_acf_diff1  \
0          1           -0.060393            0.028752        -0.521795   
1          2           -0.067574            0.056076        -0.527110   
2          3           -0.089117            0.125725        -0.534634   
3          4           -0.088535            0.158816        -0.531400   
4          5           -0.095750            0.112813        -0.524814   
5          6           -0.106356            0.199000        -0.530427   
6          7           -0.094841            0.081826        -0.519329   
7          8           -0.079998            0.077848        -0.538062   
8          9           -0.102175            0.043008        -0.512161   
9         10           -0.065051            0.101439        -0.497920   

   sumsq_acf_diff1  first_acf_diff2  sumsq_acf_diff2  seasonal_acf  
0         0.303

/Users/donaldaadithiyan/Desktop/Uni/Sem 5/Time Series-Assignment/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/donaldaadithiyan/Desktop/Uni/Sem 5/Time Series-Assignment/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [5]:
merged_train_df.dtypes

id                            int64
date                 datetime64[ns]
store_nbr                     int64
family                       object
sales                       float64
onpromotion                   int64
oil_price                   float64
city                         object
state                        object
type                         object
cluster                       int64
isHoliday                     int64
earthquake_impact             int64
salary_day_impact             int64
transactions                float64
dtype: object

In [6]:
test_df.dtypes

id                            int64
date                 datetime64[ns]
store_nbr                     int64
family                       object
onpromotion                   int64
earthquake_impact             int64
salary_day_impact             int64
isHoliday                     int64
oil_price                   float64
city                         object
state                        object
type                         object
cluster                       int64
transactions                float64
dtype: object

In [10]:
submission = pd.read_csv("my_submission_clean.csv")

# Check for NaN
print("NaN values per column:\n", submission.isna().sum())

# Check for inf
print("Infinite values per column:\n", np.isinf(submission).sum())

NaN values per column:
 id       0
sales    0
dtype: int64
Infinite values per column:
 id       0
sales    0
dtype: int64


In [9]:
import numpy as np
import pandas as pd

submission = pd.read_csv("my_submission_clean.csv")

# Find rows where sales is infinite
mask = np.isinf(submission["sales"])

# Show the offending rows
print(submission.loc[mask])


Empty DataFrame
Columns: [id, sales]
Index: []


In [8]:
import pandas as pd
import numpy as np

submission = pd.read_csv("my_submission.csv")

# Replace inf with NaN first (so ffill works)
submission["sales"].replace([np.inf, -np.inf], np.nan, inplace=True)

# Forward fill
submission["sales"].fillna(method="ffill", inplace=True)

# (Optional) if the very first row is NaN/inf, ffill won’t work — so backfill as fallback
submission["sales"].fillna(method="bfill", inplace=True)

# Save cleaned file
submission.to_csv("my_submission_clean.csv", index=False)

/var/folders/fv/7g0st0sd2gj1ztjzwt7t0s2c0000gn/T/ipykernel_60960/1686795528.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  submission["sales"].replace([np.inf, -np.inf], np.nan, inplace=True)
/var/folders/fv/7g0st0sd2gj1ztjzwt7t0s2c0000gn/T/ipykernel_60960/1686795528.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we ar

In [6]:
import pandas as pd
import numpy as np

# Load your files
test_df = pd.read_csv("test_df.csv")
submission = pd.read_csv("my_submission.csv")

# Check the range of IDs
print("▶️ Test set ID range:", test_df["id"].min(), "to", test_df["id"].max())
print("▶️ Submission ID range:", submission["id"].min(), "to", submission["id"].max())

# Also check for IDs in submission but not in test
extra_ids = set(submission["id"]) - set(test_df["id"])
if extra_ids:
    print("⚠️ IDs present in submission but not in test:", list(extra_ids)[:10], "...")
else:
    print("✅ All submission IDs are inside test_df.")


▶️ Test set ID range: 3000888 to 3029399
▶️ Submission ID range: 3000888 to 3029399
✅ All submission IDs are inside test_df.
